In [2]:
# DOCUMENTO

# Import necessary libraries for web automation, parsing, and file operations
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import os
import requests
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
import pandas as pd

# Get input URL and tag name from the user
url = input('Give the URL please')
tname = input('print the tag name')

# File naming setup
a = 'Documento'
b = '_Wdate'
csv = '.csv'
simple = a + tname + csv             # Filename for the tag counts
wd = a + tname + b + csv             # Filename for tag-date associations

# Initialize Firefox browser with WebDriver manager
driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))

# Initialize lists to store publication dates and tags
date_list = []
tag_list = []

# Date to stop scraping (YYYYMMDD format)
stop_date = 20250102

# Time delay between scrolls (in seconds)
scroll_pause_time = 2.5

# Open the initial page
driver.get(url)
time.sleep(2)
driver.maximize_window()
time.sleep(1)

# Dismiss cookie popup (click "Disagree" button)
disagree = WebDriverWait(driver, 20).until(EC.element_to_be_clickable(
    (By.XPATH, "/html/body/div[1]/div/div/div/div[2]/div/button[2]"))).click()

# Flags to control scrolling and scraping logic
flag = True
inside_flag = True

# Define scroll function to load all articles until a certain date
def scroll(driver, timeout):
    global flag
    global stop_date
    global inside_flag
    global tag_list
    global date_list
    scroll_pause_time = timeout
    nr_of_scrollings = 2
    changable_nr_of_scrollings = nr_of_scrollings

    # Internal function to check the last date shown on page
    def date_check_fun():
        global inside_flag
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')

        article_place = soup.find('div', class_="posts-list")
        dates = article_place.find_all('div', class_="item-meta")
        date = dates[-1].get_text().split(",")[0].replace("\n", "")
        dates = date.split('/')
        dates.reverse()
        date = int("".join(dates))

        # Stop scrolling if the date is older than the stop_date
        if date < stop_date:
            print('the date is smaller. Programm Must STOP')
            inside_flag = False
            return inside_flag

    # Get initial scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    # Scroll in a loop until the stop date is reached
    while inside_flag:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break  # Exit if no more scrolling is possible
        last_height = new_height

        # Every few scrolls, check the article date
        changable_nr_of_scrollings -= 1
        if changable_nr_of_scrollings == 0:
            date_check_fun()
            changable_nr_of_scrollings = nr_of_scrollings

    # Function to extract tags from an article URL
    def tag_function(inside):
        global tag_list
        inside_tag_list = []

        page = requests.get(inside)
        soup = BeautifulSoup(page.content, 'html.parser')
        tag_section = soup.find('div', class_='tags-container')

        try:
            for i in tag_section("a"):
                tag = i.get_text()
                inside_tag_list.append(tag)

            # Add main category tag (e.g., ΚΟΣΜΟΣ) to the list
            outer_tag = soup.find('div', class_='meta').find('a').get_text()
            inside_tag_list.append(outer_tag)

            tag_list.append(inside_tag_list)
        except:
            print('i have a problem. the problematic url is', inside)

    # Initial page parsing before scrolling begins
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    article_place = soup.find('div', class_="posts-list")
    articles = article_place.find_all('div', class_='meta-container')

    for article in articles:
        date = article.find('div', class_="item-meta").get_text().split(",")[0].replace("\n", "")
        dates = date.split('/')
        dates.reverse()
        date = int("".join(dates))

        if date < stop_date:
            print('the date is smaller. I need to terminate the whole programm')
            break
        else:
            article_url = article.find('h2', class_='entry-title').find('a').get('href')
            date_list.append(date)

            # Ensure full URL format
            https = "https://www.documentonews.gr"
            if article_url.find(https) < 0:
                article_url = https + article_url

            tag_function(article_url)

# Call scroll function to begin scraping
scroll(driver, scroll_pause_time)

# Create flat list of all tags from nested tag_list
new_tag_list = []
for i in range(len(tag_list)):
    for j in range(len(tag_list[i])):
        new_tag_list.append(tag_list[i][j])

# Count occurrences of each tag
res = {}
for i in new_tag_list:
    res[i] = new_tag_list.count(i)
print(res)

# Convert dictionary to list of [tag, count] pairs
resultList = list(map(list, res.items()))
print(resultList)

# Save tag counts to CSV
df_costliness = pd.DataFrame(resultList)
df_costliness.to_csv(simple, header=False)

# Print stats
print(len(tag_list))
print(len(date_list))
print('the last date collected is', date_list[-1])

# Format dates as DD.MM.YYYY
ddate_list = date_list
ttag_list = tag_list
ndlist = []
for i in ddate_list:
    i = str(i)
    y = i[:4]
    m = i[4:6]
    d = i[6:]
    ddate = d + '.' + m + '.' + y
    ndlist.append(ddate)

print(ddate_list)
print(ndlist)

# Create a list associating each tag with its article's date
n = 0
date_con_list = []
for i in ttag_list:
    n += 1
    for tag in i:
        inside = [tag, ddate_list[n - 1]]
        date_con_list.append(inside)

print(date_con_list)

# Save tag-date associations to CSV
df_connected = pd.DataFrame(date_con_list)
df_connected.to_csv(wd, header=False)

# Final summary outputs
print(len(tag_list))
print(len(date_list))
print(tag_list)
print('the last date collected is', date_list[-1])
print('the nr of the tags mentioned is', len(resultList))


the date is smaller. Programm Must STOP
the date is smaller. I need to terminate the whole programm
{'METRON ANALYSIS': 4, 'ΔΗΜΟΣΚΟΠΗΣΗ': 33, 'ΠΟΛΙΤΙΚΗ': 28, 'ΚΥΡΙΑΚΟΣ ΜΗΤΣΟΤΑΚΗΣ': 6, 'ΝΔ': 4, 'MRB': 2, 'ΑΛΕΞΗΣ ΤΣΙΠΡΑΣ': 1, 'MARC': 1, 'PODCAST': 1, 'ΝΕΑ ΔΗΜΟΚΡΑΤΙΑ': 3, 'ΠΑΣΟΚ': 4, 'ΣΥΡΙΖΑ': 2, 'ΚΟΣΜΟΣ': 5, 'ΚΥΒΕΡΝΗΣΗ ΜΗΤΣΟΤΑΚΗ': 4, 'ΤΕΜΠΗ': 4, 'ΕΥΡΩΠΑΪΚΗ ΕΝΩΣΗ': 1, 'ΚΑΝΑΔΑΣ': 1, 'ΕΓΚΛΗΜΑ ΤΕΜΠΗ': 12, 'ALCO': 2, 'PROTAGON': 2, 'ΣΥΛΛΑΛΗΤΗΡΙΑ': 1, 'ΓΙΩΡΓΟΣ ΚΥΡΤΣΟΣ': 1, 'ΑΚΡΙΒΕΙΑ': 1, 'ΚΚΕ': 1, 'ΜέΡΑ25': 1, 'ΠΛΕΥΣΗ ΕΛΕΥΘΕΡΙΑΣ': 1, 'ΦΩΝΗ ΛΟΓΙΚΗΣ': 1, 'Ινστιτούτο Αλέξη Τσίπρα': 1, 'ΣΚΑΝΔΑΛΟ ΥΠΟΚΛΟΠΩΝ': 1, 'ΒΡΕΤΑΝΙΑ': 2, 'ΝΑΪΤΖΕΛ ΦΑΡΑΤΖ': 2, 'ΕΚΛΗΜΑ ΤΕΜΠΗ': 1, 'ΚΥΡΙΑΚΟΣ ΜΗΤΣΤΟΤΑΚΗΣ': 1, 'ΝΤΟΝΑΛΝΤ ΤΡΑΜΠ': 1, 'ΕΡΓΑΤΙΚΟΙ': 1}
[['METRON ANALYSIS', 4], ['ΔΗΜΟΣΚΟΠΗΣΗ', 33], ['ΠΟΛΙΤΙΚΗ', 28], ['ΚΥΡΙΑΚΟΣ ΜΗΤΣΟΤΑΚΗΣ', 6], ['ΝΔ', 4], ['MRB', 2], ['ΑΛΕΞΗΣ ΤΣΙΠΡΑΣ', 1], ['MARC', 1], ['PODCAST', 1], ['ΝΕΑ ΔΗΜΟΚΡΑΤΙΑ', 3], ['ΠΑΣΟΚ', 4], ['ΣΥΡΙΖΑ', 2], ['ΚΟΣΜΟΣ', 5], ['ΚΥΒΕΡΝΗΣΗ ΜΗΤΣΟΤΑΚΗ', 4], ['ΤΕΜΠΗ', 4],